In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import numpy as np
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt

In [2]:
### Define column headers for csv export
columns_21 = ['Event', 'School', 'TEA', 'City', 'Directors', 'Conference', 'Classification', 'Year', 'ID', 
    'Stage Judge 1', 'Stage Judge 2', 'Stage Judge 3', 'Stage Final', 
    'SR Judge 1', 'SR Judge 2', 'SR Judge 3', 'SR Final', 'Award', 
    'Selection 1', 'Selection 2', 'Selection 3', 'Date', 'Region', 'cj1', 'cj2', 'cj3', 'srj1', 'srj2', 'srj3']
columns_22 = ['Event', 'School', 'TEA', 'City', 'Directors', 'Conference', 'Classification', 'Year', 'ID', 
    'Stage Judge 1', 'Stage Judge 2', 'Stage Judge 3', 'Stage Final', 
    'SR Judge 1', 'SR Judge 2', 'SR Judge 3', 'SR Final', 'Award', 
    'Selection 1', 'Selection 2', 'Selection 3', 'Date', 'Region', 'cj1', 'cj2', 'cj3', 'srj1', 'srj2', 'srj3', 'oops']

## Group the CSVs together, and fix rows with Accompanist error

In [3]:
# list of merged files returned
files = glob.glob("full_run/*.csv")

# joining files with concat and read_csv
df = pd.concat(map(pd.read_csv, files), ignore_index=True)
df = pd.DataFrame(df)

# drop unnamed column
df.drop(columns=['Unnamed: 0'], inplace=True)

# name the columns
df.columns = columns_22

In [4]:
# select rows with acc column 
oops_df = df[df['oops'].notnull()]

# delete the acc columns and fix column names
oops_fix_df = oops_df.drop(columns=['Conference'])
oops_fix_df.columns = columns_21

# drop acc rows from df
df = df.loc[df['oops'].isnull() == True]

# add fixed df to df
df = pd.concat([df, oops_fix_df], ignore_index=True)

# drop oops column
df.drop(columns=['oops'], inplace=True)

# drop rows where Event contains '9'
df = df[df['Event'].str.contains('9') == False]

df.describe()

Event              School    TEA          City  \
count               60803               60803  60803         60803   
unique                  1                6236   2373          1498   
top      100-Concert Band   Allen High School   TEA:   San Antonio   
freq                60803                 112  56877          2882   

            Directors Conference Classification   Year     ID  Stage Judge 1  \
count           60803      60803          60803  60803  60803          60803   
unique          24456         28             39    210  60631             13   
top      Joe Martinez         CC       Varsity    2019      1              1   
freq               29      14895          31202   3274     78          25874   

        ...                       Selection 2                Selection 3  \
count   ...                             60803                      60803   
unique  ...                              8975                      11894   
top     ...   Colliding Visions  (Balmages/ )   Moscow, 1941  (Balmages)   
freq    ...                               422                        446   

                             Date     Region               cj1  \
count                       60803      60803             60803   
unique                        499        155              1116   
top     DATE of EVENT 04/17/2018   Region: 8  1. Keith Bearden   
freq                          714       3135               709   

                      cj2              cj3             srj1              srj2  \
count               60803            60803            60803             60803   
unique               1219             1196             1230              1344   
top     2. Cindy Lansford  3. Randy Vaughn  1. Phil Anthony  2. Tye Ann Payne   
freq                  463              843              448               341   

                  srj3  
count            60625  
unique            1203  
top     3. Rick Yancey  
freq               736  

[4 rows x 29 columns]

In [5]:
# select rows where selection 3 is null
df_clean = df
df_clean = df_clean[df_clean['Selection 3'].isnull() == False]
len(df_clean)

60803

In [6]:
# Trim whitespace, double spaces, commas, and periods from selections
selection_columns = ['Selection 1', 'Selection 2', 'Selection 3']

for i in selection_columns:
    df_clean[i] = df_clean[i].str.strip()
    df_clean[i] = df_clean[i].str.replace('  ', ' ', regex=False)
    df_clean[i] = df_clean[i].str.replace(',', '', regex=False)
    df_clean[i] = df_clean[i].str.replace('.', '', regex=False)

# Trim whitespace from classification column
df_clean['Classification'] = df_clean['Classification'].str.strip()

# Remove composer/arranger information from selections
for column in selection_columns:
    # remove all inside parenthesis
    df_clean[column] = df_clean[column].str.replace('\(.*\)', '', regex=True)
    # trim whitespace
    df_clean[column] = df_clean[column].str.strip()

In [7]:
# drop rows where conference contains 'Acc'
df_clean = df_clean[df_clean['Conference'].str.contains('Acc') == False]


In [8]:
# Fix names of some conferences
df_clean['Conference'] = df_clean['Conference'].replace('2C', 'CC')
df_clean['Conference'] = df_clean['Conference'].replace('4A', 'AAAA')
df_clean['Conference'] = df_clean['Conference'].replace('cc', 'CC')
df_clean['Conference'] = df_clean['Conference'].replace('1C', 'C')

In [9]:
# Make df_clean column integers
df_clean['Year'] = df_clean['Year'].astype(int)

In [10]:
print(df_clean['Conference'].unique())
print(df_clean['Classification'].unique())
print(df_clean['Year'].unique())

['Varsity ' 'CC' 'AAA' 'C' 'CCC' 'AAAA' 'AA' 'A' 'AAAAA' 'Non-Varsity '
 'Sub Non-Varsity ' 'Sub Non-Varsity B' 'BBB' 'Non-Varsity A' 'BB' 'B'
 'Var-Composite ' ' ' 'Non-Varsity C' 'NVar-Composite ' 'Var-Combined '
 'AAAAAA']
['2005' 'Varsity' 'Non-Varsity' 'Non-Varsity A' 'Non-Varsity B' 'Combined'
 'Sub Non-Varsity' 'Sub Non-Varsity C' 'Sub Non-Varsity B'
 'Sub Non-Varsity A' 'Non-Varsity C' 'Non-Varsity F' 'Sub Non-Varsity D'
 'Sub Non-Varsity E' 'Non-Varsity E' '2006' 'Non-Varsity D' '2008'
 'Var-Combined' 'Var-Composite' '' '2009' 'NVar-Composite' '2010' '2011'
 '2012' 'NVar-Combined' '2013' 'Sub Non-Varsity F' '2014' 'Varsity A'
 'Varsity C' 'Varsity B' '2015' 'Sub Non-Varsity G' 'Var-Composite A'
 'Sub Non-Varsity H' 'Sub Non-Varsity I' 'NVar-Composite A']
[ 17246   2005  14077  11197  11109  17900  16404  17069  17328  13344
  15094  13014  17315  17648  18475  18405  17673  14770  13205  13902
  15470  15934  13485  16213  15350  15679  15324  15314  15307  15306
  15299  1528

In [11]:
judging_columns = ['Stage Judge 1', 'Stage Judge 2', 'Stage Judge 3', 'Stage Final', 'SR Judge 1', 
                'SR Judge 2', 'SR Judge 3', 'SR Final']
numbers = ['1', '2', '3', '4', '5']

for n in numbers:
    for j in judging_columns:
        df_clean.loc[df_clean[j] == n, j] = int(n)




In [12]:
for j in judging_columns:
    print(df_clean[j].unique())

[' ' 2 1 3 4 'DNA' 5 'DQ']
[' ' 2 1 4 3 'DNA' 5 'DQ']
[' ' 2 3 1 4 'DNA' 'DQ' 5]
[' ' 2 1 3 4 'DNA' 5 'DQ']
[' ' 1 2 3 4 5 'DNA' 'DQ']
[' ' 1 2 3 4 'DNA' 5 'DQ']
[' ' 1 2 3 4 'DNA' 5 'DQ']
[' ' 1 2 3 4 'DNA' 5 'TD' 'TRC' 'C' 'DQ' 'PLQ' 'TRO' 'RM1' 'RMD' 'RM2'
 'RMC' 'A' '-' 11 'SWA' 'NAN' 'B' 'D' 0]


In [13]:
# Trim whitespace, double spaces, commas, and periods from selections
selection_columns = ['Selection 1', 'Selection 2', 'Selection 3']

for i in selection_columns:
    df_clean[i] = df_clean[i].str.strip()
    df_clean[i] = df_clean[i].str.replace('  ', ' ', regex=False)
    df_clean[i] = df_clean[i].str.replace(',', '', regex=False)
    df_clean[i] = df_clean[i].str.replace('.', '', regex=False)

# Trim whitespace from classification column
df_clean['Classification'] = df_clean['Classification'].str.strip()

# Remove composer/arranger information from selections
for column in selection_columns:
    # remove all inside parenthesis
    df_clean[column] = df_clean[column].str.replace('\(.*\)', '', regex=True)
    # trim whitespace
    df_clean[column] = df_clean[column].str.strip()


In [14]:
df_clean.columns = df_clean.columns.str.strip()

In [15]:
# Drop Event and TEA column
df = df.drop(columns=['Event', 'TEA'])
print(len(df))

60803


In [16]:
# Remove DNA, DQ
for i in judging_columns:
    df_clean = df_clean[df_clean[i] != 'DNA']
    df_clean = df_clean[df_clean[i] != 'DQ']

# Convert blanks to nans
for i in judging_columns:
    df_clean[i] = df_clean[i].replace(['', ' '], np.nan)

print(len(df_clean))

60269


In [17]:
# Average together Stage Judge 1, 2, and 3 into a new column

df_clean['Stage Average'] = (df_clean['Stage Judge 1'] + df_clean['Stage Judge 2'] + df_clean['Stage Judge 3']) / 3

In [18]:
# sort df_clean by year
df_clean.sort_values(by=['Year'], inplace=True)

# drop Event column
df_clean = df_clean.drop(columns=['Event'])

# drop TEA column
df_clean = df_clean.drop(columns=['TEA'])

# drop ID column
df_clean = df_clean.drop(columns=['ID'])




In [19]:
# import all of the pml csv files into a dataframe
files = glob.glob("csv_files/pml/*.csv")
pml_df = pd.concat([pd.read_csv(f) for f in files], ignore_index=True)
# convert pml_df to csv
pml_df.to_csv("csv_files/pml_output.csv")

In [20]:
df_clean['Date'] = df_clean['Date'].str.replace('DATE of EVENT ', '')
judge_name_columns = ['cj1', 'cj2', 'cj3', 'srj1', 'srj2', 'srj3']
for i in judge_name_columns:
    df_clean[i] = df_clean[i].str.replace('1. ', '', regex=False)
    df_clean[i] = df_clean[i].str.replace('2. ', '', regex=False)
    df_clean[i] = df_clean[i].str.replace('3. ', '', regex=False)
    # all lowercase
    df_clean[i] = df_clean[i].str.lower()

In [21]:
# for each judge name column, remove everything after the first comma
for i in judge_name_columns:
    #df_clean[i] = df_clean[i].str.split(',', expand=True)[0]
    #df_clean[i] = df_clean[i].str.split('-', expand=True)[0]
    # trim whitespace
    df_clean[i] = df_clean[i].str.strip()

In [22]:
all_columns = df_clean.columns
# trim whitespace from all columns
for i in all_columns:
    try:
        df_clean[i] = df_clean[i].str.strip()
    except:
        pass

In [23]:
# drop rows where School = 0
df_clean = df_clean[df_clean['School'] != '0']

In [24]:
# get rows where year is greater than 3000
df_clean_yr_error = df_clean[df_clean['Year'] > 3000]

# drop rows where year is greater than 3000
df_clean = df_clean[df_clean['Year'] < 3000]

df_clean_yr_error['Year'] = df_clean_yr_error['Classification']
df_clean_yr_error['Classification'] = df_clean_yr_error['Conference']

# fill conference column with blanks
df_clean_yr_error['Conference'] = ''


In [25]:
# combine df_clean and df_clean_yr_error
df_clean = pd.concat([df_clean, df_clean_yr_error], ignore_index=True)

In [26]:
print(df_clean['Conference'].unique())
print(df_clean['Classification'].unique())
print(df_clean['Year'].unique())

['AAAAA' 'A' 'AA' 'AAA' 'AAAA' 'C' 'CC' 'CCC' 'BBB' 'BB' 'B' 'AAAAAA' '']
['Non-Varsity' 'Varsity' 'Sub Non-Varsity' 'Non-Varsity A'
 'Sub Non-Varsity B' 'Sub Non-Varsity A' 'Sub Non-Varsity C'
 'Sub Non-Varsity E' 'Sub Non-Varsity D' 'Non-Varsity B' 'Non-Varsity E'
 'Non-Varsity C' 'Combined' 'Non-Varsity F' 'Non-Varsity D'
 'Var-Composite' 'Var-Combined' '' 'NVar-Composite' 'NVar-Combined'
 'Sub Non-Varsity F' 'Varsity C' 'Varsity B' 'Varsity A'
 'Sub Non-Varsity G' 'Var-Composite A' 'Sub Non-Varsity H'
 'Sub Non-Varsity I' 'NVar-Composite A']
[2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018
 2019 2020 2021 2022 '2008' '2009' '2010' '2011' '2012' '2013' '2014'
 '2015']


In [27]:
df_clean.dtypes

School             object
City               object
Directors          object
Conference         object
Classification     object
Year               object
Stage Judge 1     float64
Stage Judge 2     float64
Stage Judge 3     float64
Stage Final       float64
SR Judge 1        float64
SR Judge 2        float64
SR Judge 3        float64
SR Final           object
Award              object
Selection 1        object
Selection 2        object
Selection 3        object
Date               object
Region             object
cj1                object
cj2                object
cj3                object
srj1               object
srj2               object
srj3               object
Stage Average     float64
dtype: object

In [28]:
# for each column, remove all text after ' - '
for i in judge_name_columns:
    df_clean[i] = df_clean[i].str.split(' - ', expand=True)[0]
    # trim whitespace
    df_clean[i] = df_clean[i].str.strip()

df_clean


School           City                    Directors  \
0      James Bowie High School      Arlington                  Larry Brown   
1           Munday High School         Munday            Rodney D. Bennett   
2         Holliday High School       Holliday            Melanie Hadderton   
3          Decatur High School        Decatur                 Doug Fulwood   
4        City View High School  Wichita Falls            Terah Kay Shawver   
...                        ...            ...                          ...   
60225              Alvarado MS       Alvarado      Kelli Bahner / Joe Gunn   
60226              Alvarado MS       Alvarado      Kelli Bahner / Joe Gunn   
60227      Del Rio High School        Del Rio                 Daniel White   
60228    Stinson Middle School    San Antonio  Kevin Leman / Alex Melendez   
60229      Del Rio High School        Del Rio                 Daniel White   

      Conference   Classification  Year  Stage Judge 1  Stage Judge 2  \
0          AAAAA      Non-Varsity  2005            1.0            1.0   
1              A          Varsity  2005            1.0            1.0   
2             AA          Varsity  2005            2.0            2.0   
3            AAA          Varsity  2005            1.0            2.0   
4             AA          Varsity  2005            1.0            1.0   
...          ...              ...   ...            ...            ...   
60225                 Non-Varsity  2014            1.0            3.0   
60226                     Varsity  2014            1.0            1.0   
60227             Sub Non-Varsity  2014            1.0            1.0   
60228                 Non-Varsity  2014            1.0            1.0   
60229                 Non-Varsity  2015            2.0            2.0   

       Stage Judge 3  Stage Final  ...               Selection 3        Date  \
0                1.0          1.0  ...                   Festivo  04/06/2005   
1                2.0          1.0  ...           Flight of Valor  03/10/2005   
2                3.0          2.0  ...                    Pablo!  03/10/2005   
3                2.0          2.0  ...    West Highlands Sojourn  03/10/2005   
4                1.0          1.0  ...           Canterbury Walk  03/10/2005   
...              ...          ...  ...                       ...         ...   
60225            1.0          2.0  ...  DATE of EVENT 03/25/2014   Region: 7   
60226            1.0          1.0  ...  DATE of EVENT 03/25/2014   Region: 7   
60227            1.0          1.0  ...  DATE of EVENT 04/10/2014  Region: 11   
60228            1.0          1.0  ...  DATE of EVENT 04/16/2014  Region: 11   
60229            2.0          1.0  ...  DATE of EVENT 04/07/2015  Region: 11   

                  Region                    cj1               cj2  \
0              Region: 5           richard bass    joe frank, jr.   
1              Region: 2             mike glaze   richard herrera   
2              Region: 2             mike glaze   richard herrera   
3              Region: 2             mike glaze   richard herrera   
4              Region: 2             mike glaze   richard herrera   
...                  ...                    ...               ...   
60225  1. Bryan Bronstad  christine cumberledge  james marioneaux   
60226  1. Bryan Bronstad  christine cumberledge  james marioneaux   
60227    1. John Faraone            bob whipkey  rogerio olivarez   
60228   1. Penny Compton          kim rosenberg       javier vera   
60229   1. Jeff Lightsey            ronnie rios      james snider   

                    cj3              srj1            srj2          srj3  \
0          rodney klett      george jones     tom neugent   marion west   
1            will burks       harold bufe       mack bibb  june bearden   
2            will burks       harold bufe       mack bibb  june bearden   
3            will burks       harold bufe       mack bibb  june bearden   
4            will burks       harold bufe    

In [29]:
# count the unique values in cj1
lolcody = df_clean['cj1'].value_counts()

In [32]:
for each in judge_name_columns:
    df_clean[each] = df_clean[each].str.replace(', lubbock', '', regex=False)
    df_clean[each] = df_clean[each].str.replace("-harlingen", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", san antonio", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", sharyland", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", austin", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", moore", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", mission", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", edinburg north hs, edinburg cisd", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", edinburg north hs", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", edinburg cisd", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", lake travis", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", robstown hs", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", robstown", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", h.e.b.", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", duncanville", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", concert", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", sr", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", roma isd", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", roma", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", retired", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", corpus christi", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", la feria hs", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", la feria", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", iii", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", orange grove", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", weslaco", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", chair", '', regex=False)
    df_clean[each] = df_clean[each].str.replace("(chair)", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", director fine arts", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", director of fine arts", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", tomball isd", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", baytown", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", ret.", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", edinburg hs", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", edinburg", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", juarez-lincoln hs", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", la joya isd", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", la joya", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", alvarado", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", mcallen isd", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", mcallen", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", united isd", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", laredo", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", mcqueeny", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", seguin", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", mission cisd", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", connally hs", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", el paso", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", el paso isd", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", brownsville isd", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", brownsville", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", inst.musicadv.", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", grulla hs", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", rio grande city isd", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", rio grande city", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", midway hs", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", woodway", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", tuloso-midway hs", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", ac blunt ms", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", aransas pass", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", leander", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", falfurrias hs", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", falfurrias", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", kingsville", '', regex=False)
    df_clean[each] = df_clean[each].str.replace("-retired", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", weslaco isd", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", fine arts administrator", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(",la joya isd", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", aledo", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", canyon", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", p.s.j.a.", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", odem hs", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", c.o. wilson ms", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", garland", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", houston", '', regex=False)
    df_clean[each] = df_clean[each].str.replace("-banda", '', regex=False)
    df_clean[each] = df_clean[each].str.replace("*", '', regex=False)
    df_clean[each] = df_clean[each].str.replace("--", '', regex=False)
    df_clean[each] = df_clean[each].str.replace("1", '', regex=False)
    df_clean[each] = df_clean[each].str.replace("2", '', regex=False)
    df_clean[each] = df_clean[each].str.replace("3", '', regex=False)
    df_clean[each] = df_clean[each].str.replace("4", '', regex=False)
    df_clean[each] = df_clean[each].str.replace("5", '', regex=False)
    df_clean[each] = df_clean[each].str.replace("6", '', regex=False)
    df_clean[each] = df_clean[each].str.replace("xxx", '', regex=False)
    df_clean[each] = df_clean[each].str.replace("virginia osolvsky", 'virginia olsovsky', regex=False)
    df_clean[each] = df_clean[each].str.replace("virginia osovsky", 'virginia olsovsky', regex=False)
    df_clean[each] = df_clean[each].str.replace("virginian olsovsky", 'virginia olsovsky', regex=False)
    df_clean[each] = df_clean[each].str.replace("villareal", 'villarreal', regex=False)
    df_clean[each] = df_clean[each].str.replace("-flour bluff", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(" d. ", ' ', regex=False)
    df_clean[each] = df_clean[each].str.replace("wallace diefolf", 'wallace dierolf', regex=False)
    df_clean[each] = df_clean[each].str.replace("walace dierolf", 'wallace dierolf', regex=False)
    df_clean[each] = df_clean[each].str.replace("unknown", '', regex=False)
    df_clean[each] = df_clean[each].str.replace("ty ann payne", 'tye ann payne', regex=False)
    df_clean[each] = df_clean[each].str.replace("tye ann payne", 'tye payne', regex=False)
    df_clean[each] = df_clean[each].str.replace("-bastrop", '', regex=False)
    df_clean[each] = df_clean[each].str.replace("(10th)", '', regex=False)
    df_clean[each] = df_clean[each].str.replace("(11th)", '', regex=False)
    df_clean[each] = df_clean[each].str.replace("allmany", 'almany', regex=False)
    df_clean[each] = df_clean[each].str.replace("mcelory", 'mcelroy', regex=False)
    df_clean[each] = df_clean[each].str.replace("knolficek", 'knoflicek', regex=False)
    df_clean[each] = df_clean[each].str.replace("knloficek", 'knoflicek', regex=False)
    df_clean[each] = df_clean[each].str.replace("knofllicek", 'knoflicek', regex=False)
    df_clean[each] = df_clean[each].str.replace("tom herrington", 'tom harrington', regex=False)
    df_clean[each] = df_clean[each].str.replace(" (chrmn.)", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(" (chmn)", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(" (chmn.)", '', regex=False)
    df_clean[each] = df_clean[each].str.replace("clearwarter", 'clearwater', regex=False)
    df_clean[each] = df_clean[each].str.replace(" .j ", ' ', regex=False)
    df_clean[each] = df_clean[each].str.replace("tim edens", 'tim edins', regex=False)
    df_clean[each] = df_clean[each].str.replace("tim andersen", 'tim anderson', regex=False)
    df_clean[each] = df_clean[each].str.replace("terri brockway", 'teri brockway', regex=False)
    df_clean[each] = df_clean[each].str.replace("tbd/", '', regex=False)
    df_clean[each] = df_clean[each].str.replace("tammy fedenych", 'tammy fedynich', regex=False)
    df_clean[each] = df_clean[each].str.replace("tammy fedinich", 'tammy fedynich', regex=False)
    df_clean[each] = df_clean[each].str.replace("tammy fednmich", 'tammy fedynich', regex=False)
    df_clean[each] = df_clean[each].str.replace("susan meyer-patterson", 'susan patterson', regex=False)
    df_clean[each] = df_clean[each].str.replace("susan meyer patterson", 'susan patterson', regex=False)
    df_clean[each] = df_clean[each].str.replace("-houston", '', regex=False)
    df_clean[each] = df_clean[each].str.replace("-pearsall", '', regex=False)
    df_clean[each] = df_clean[each].str.replace("-psja", '', regex=False)
    df_clean[each] = df_clean[each].str.replace("-corpus christi", '', regex=False)
    df_clean[each] = df_clean[each].str.replace("-keller", '', regex=False)
    df_clean[each] = df_clean[each].str.replace("steel, jason", 'steele, jason', regex=False)
    df_clean[each] = df_clean[each].str.replace("maudlin", 'mauldin', regex=False)
    df_clean[each] = df_clean[each].str.replace("stacy claek", 'stacy clark', regex=False)
    df_clean[each] = df_clean[each].str.replace("stacy clark", 'stacey clark', regex=False)
    df_clean[each] = df_clean[each].str.replace("sr #1", '', regex=False)
    df_clean[each] = df_clean[each].str.replace("sr #2", '', regex=False)
    df_clean[each] = df_clean[each].str.replace("sr #3", '', regex=False)
    df_clean[each] = df_clean[each].str.replace("4/11/17", '', regex=False)
    df_clean[each] = df_clean[each].str.replace("4/13/17", '', regex=False)
    df_clean[each] = df_clean[each].str.replace("see region 21 web site", '', regex=False)
    df_clean[each] = df_clean[each].str.replace("see original event", '', regex=False)
    df_clean[each] = df_clean[each].str.replace("see original contest", '', regex=False)
    df_clean[each] = df_clean[each].str.replace("see original evenrt", '', regex=False)
    df_clean[each] = df_clean[each].str.replace("sheppard", 'shepherd', regex=False)
    df_clean[each] = df_clean[each].str.replace("-beaumont", '', regex=False)
    df_clean[each] = df_clean[each].str.replace("sandy bow brunskill", 'sandra brunskill', regex=False)
    df_clean[each] = df_clean[each].str.replace("sandy brunskill", 'sandra brunskill', regex=False)
    df_clean[each] = df_clean[each].str.replace("sandra bow brunskill", 'sandra brunskill', regex=False)
    df_clean[each] = df_clean[each].str.replace("sandra bow-brunskill", 'sandra brunskill', regex=False)
    df_clean[each] = df_clean[each].str.replace("rylon guidory", 'rylon guidry', regex=False)
    df_clean[each] = df_clean[each].str.replace("-austin", '', regex=False)
    df_clean[each] = df_clean[each].str.replace("ryan straten", 'ryan stratten', regex=False)
    df_clean[each] = df_clean[each].str.replace("-mabank", '', regex=False)
    df_clean[each] = df_clean[each].str.replace("rusty honeycutt", 'rustin honeycutt', regex=False)
    df_clean[each] = df_clean[each].str.replace("barerra", 'barrera', regex=False)
    df_clean[each] = df_clean[each].str.replace("costellano", 'castellano', regex=False)
    df_clean[each] = df_clean[each].str.replace(" m. ", ' ', regex=False)
    df_clean[each] = df_clean[each].str.replace(",san antonio", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(", frisco", '', regex=False)
    df_clean[each] = df_clean[each].str.replace("/poteet", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(" (8th)", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(" (9th)", '', regex=False)
    df_clean[each] = df_clean[each].str.replace("-spring", '', regex=False)
    df_clean[each] = df_clean[each].str.replace("margarit", 'margaret', regex=False)
    df_clean[each] = df_clean[each].str.replace("nv-", '', regex=False)
    df_clean[each] = df_clean[each].str.replace("/v-", '/', regex=False)
    df_clean[each] = df_clean[each].str.replace("//", '/', regex=False)
    #df_clean[each] = df_clean[each].str.replace("^x?", '', regex=True)
    df_clean[each] = df_clean[each].str.replace("^tba?", '', regex=True)
    df_clean[each] = df_clean[each].str.replace("^tbd?", '', regex=True)
    df_clean[each] = df_clean[each].str.replace("- -", '', regex=False)
    df_clean[each] = df_clean[each].str.replace("-plano", '', regex=False)
    df_clean[each] = df_clean[each].str.replace(" .k ", ' ', regex=False)
    df_clean[each] = df_clean[each].str.replace("bene davis ", 'ben davis', regex=False)
    df_clean[each] = df_clean[each].str.replace("benny davis", 'ben davis', regex=False)
    df_clean[each] = df_clean[each].str.replace("see region website", '', regex=False)
    df_clean[each] = df_clean[each].str.replace("see region web site", '', regex=False)
    df_clean[each] = df_clean[each].str.replace("rick yancy", 'rick yancey', regex=False)
    df_clean[each] = df_clean[each].str.replace("phillip alvarado", 'phil alvarado', regex=False)
    #df_clean[each] = df_clean[each].str.replace("^d?", '', regex=True)
    #df_clean[each] = df_clean[each].str.replace("^f?", '', regex=True)
    #df_clean[each] = df_clean[each].str.replace("^judge?", '', regex=True)
    #df_clean[each] = df_clean[each].str.replace("^b?", '', regex=True)
    #df_clean[each] = df_clean[each].str.replace("^c?", '', regex=True)
    #df_clean[each] = df_clean[each].str.replace("^a?", '', regex=True)
    #df_clean[each] = df_clean[each].str.replace("^e?", '', regex=True)
    #df_clean[each] = df_clean[each].str.replace("^t?", '', regex=True)

    df_clean[each] = df_clean[each].str.strip()

















In [33]:
# convert to csv
df_clean.to_csv("csv_files/full_output.csv")